### Demo Preparation Notebook

**Please Note**: This notebook and demo are NOT intended to be used as learning materials. To gain
a thorough understanding of the DataJoint workflow for extracellular electrophysiology, please
see the [`tutorial`](./tutorial.ipynb) notebook.

In [ ]:
# Runs in about 45s
import datajoint as dj
import datetime
from workflow_array_ephys.pipeline import subject, session, probe, ephys
from element_array_ephys import ephys_report


In [ ]:
subject.Subject.insert1(
    dict(
        subject='subject5',
        subject_birth_date='2023-01-01',
        sex='U'
    )
)

In [ ]:
session_key = dict(subject='subject5', 
                   session_datetime=datetime.datetime.now())

session.Session.insert1(session_key)

session.SessionDirectory.insert1(
    dict(
        session_key, 
        session_dir='subject5/session1'
    )
)

In [ ]:
probe.Probe.insert1(
    dict(probe="714000838", 
         probe_type="neuropixels 1.0 - 3B")
)

ephys.ProbeInsertion.insert1(
    dict(
        session_key,
        insertion_number=1,
        probe="714000838",
    )
)

In [ ]:
populate_settings = {"display_progress": True}

ephys.EphysRecording.populate(**populate_settings)

In [ ]:
kilosort_params = {
    "fs": 30000,
    "fshigh": 150,
    "minfr_goodchannels": 0.1,
    "Th": [10, 4],
    "lam": 10,
    "AUCsplit": 0.9,
    "minFR": 0.02,
    "momentum": [20, 400],
    "sigmaMask": 30,
    "ThPr": 8,
    "spkTh": -6,
    "reorder": 1,
    "nskip": 25,
    "GPU": 1,
    "Nfilt": 1024,
    "nfilt_factor": 4,
    "ntbuff": 64,
    "whiteningRange": 32,
    "nSkipCov": 25,
    "scaleproc": 200,
    "nPCs": 3,
    "useRAM": 0,
}

ephys.ClusteringParamSet.insert_new_params(
    clustering_method="kilosort2",
    paramset_idx=1,
    params=kilosort_params,
    paramset_desc="Spike sorting using Kilosort2",
)

In [ ]:
ephys.ClusteringTask.insert1(
    dict(
        session_key,
        insertion_number=1,
        paramset_idx=1,
        task_mode='load', # load or trigger
        clustering_output_dir="subject5/session1/probe_1/kilosort2-5_1"
    )
)

ephys.Clustering.populate(**populate_settings)

In [ ]:
clustering_key = (ephys.ClusteringTask & session_key).fetch1('KEY')
ephys.Curation().create1_from_clustering_task(clustering_key)

In [ ]:
# Runs in about 12m
ephys.CuratedClustering.populate(**populate_settings)
ephys.WaveformSet.populate(**populate_settings)
ephys_report.ProbeLevelReport.populate(**populate_settings)
ephys_report.UnitLevelReport.populate(**populate_settings)

### Drop schemas
- Schemas are not typically dropped in a production workflow with real data in it.
- At the developmental phase, it might be required for the table redesign.
- When dropping all schemas is needed, the following is the dependency order.

In [ ]:
def drop_databases(databases):
    import pymysql.err
    conn = dj.conn()

    with dj.config(safemode=False):
        for database in databases:
            schema = dj.Schema(f'{dj.config["custom"]["database.prefix"]}{database}')
            while schema.list_tables():
                for table in schema.list_tables():
                    try:
                        conn.query(f"DROP TABLE `{schema.database}`.`{table}`")
                    except pymysql.err.OperationalError:
                        print(f"Can't drop `{schema.database}`.`{table}`. Retrying...")
            schema.drop()

# drop_databases(databases=['analysis', 'trial', 'event', 'ephys_report', 'ephys', 'probe', 'session', 'subject', 'project', 'lab'])
